In [1]:
from PIL import Image, ImageOps
import glob
import numpy as np
import onnxruntime as ort
import pandas as pd
import torch
import cv2
import time
import torchvision
import math

In [2]:
target_size = 640
path = glob.glob('*.jpg')
# img_list = []
# for img in path:
#     image = Image.open(path[0])
#     image.thumbnail((target_size, target_size))
#     padded_img = ImageOps.pad(image, (target_size, target_size))
#     img_np = np.array(padded_img).astype(np.float32)/255.0
#     img_np = img_np.transpose([2, 0, 1])
#     img_np = np.expand_dims(img_np, axis=0)
#     img_list.append(img_np)

# np.array(img_list).shape

(1, 1, 3, 640, 640)

In [3]:
session = ort.InferenceSession('best.onnx')
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

result = session.run([output_name], {input_name: img_list[0]})

In [4]:
"""
yolov5.githubの*.pyの関数を使用した場合
"""
# import torch
# from utils.general import non_max_suppression

# # モデルからの出力
# predictions = result[0]  # ここで result は session.run からの出力

# # NumPy配列からPyTorchテンソルへの変換
# predictions_tensor = torch.from_numpy(predictions).float() 

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# predictions_tensor = predictions_tensor.to(device)

# # 非最大抑制(NMS)を適用
# conf_thres = 0.6  # 信頼度閾値
# iou_thres = 0.45  # IOU閾値
# classes = None  # 特定のクラスにフィルタリングする場合は、クラスIDのリスト
# agnostic_nms = False  # クラスに依存しないNMSを使用するかどうか
# max_det = 100  # 画像あたりの最大検出数

# # NMS処理
# nms_predictions = non_max_suppression(predictions_tensor, conf_thres, iou_thres, classes=classes, agnostic=agnostic_nms, max_det=max_det)

# # 処理後の検出を処理
# for i, det in enumerate(nms_predictions):  # 画像ごとの検出結果をループ
#     if len(det):
#         # det[:, :4] はバウンディングボックスの座標(x1, y1, x2, y2)
#         # det[:, 4] はオブジェクトネススコア
#         # det[:, 5] は最も可能性の高いクラスID
#         print(f"画像 {i}: 検出されたオブジェクトの数: {len(det)}")
#         for *xyxy, conf, cls in det:
#             print(f"    座標: {xyxy}, 信頼度: {conf}, クラス: {cls}")

画像 0: 検出されたオブジェクトの数: 4
    座標: [tensor(483.90857), tensor(543.44012), tensor(529.23657), tensor(598.27289)], 信頼度: 0.9582321643829346, クラス: 0.0
    座標: [tensor(389.48328), tensor(70.23535), tensor(432.98383), tensor(127.41638)], 信頼度: 0.9482046961784363, クラス: 0.0
    座標: [tensor(431.67334), tensor(69.08841), tensor(474.15851), tensor(127.48247)], 信頼度: 0.9455069899559021, クラス: 1.0
    座標: [tensor(473.43066), tensor(67.75636), tensor(515.72369), tensor(125.53108)], 信頼度: 0.9404840469360352, クラス: 1.0


## スクラッチで実装

In [40]:
#------------
# 
#------------

#runの引数
input_file = path[0]
#session

#--------------

# run()
max_det = 1000  # 画像あたりの最大bbox数
conf_thres = 0.6  # 信頼値の閾値
iou_thres = 0.45  # NMS IOU の閾値
imgsz = (640, 640) # インプットの画像サイズ
agnostic_nms = False # 異なるbboxの重なりをマージするか
hide_conf = False # 信頼度スコアの非表示をするか
stride = 32 

# onnxに保存されているメタデータ -> {'names': "{0: '押印', 1: '未押印'}", 'stride': '32'}
meta = session.get_modelmeta().custom_metadata_map

# metadataがあればをアンパックする
if 'stride' in meta:
    stride, names = int(meta['stride']), eval(meta['names'])

# 16ビット浮動小数点精度の使用
fp16 = False

# 画像のサイズをstrideの倍数にする
imgsz = [max(math.ceil(x / stride) * stride, 0) for x in imgsz]
output_name = session.get_outputs()[0].name

In [210]:
# ---- 画像の前処理 ----
np_img = cv2.imread(input_file)

# 画像のリサイズ
img = letterbox(np_img, [640, 640])[0]

# (H,W,C) -> (C, H, W)
# BGR -> RGB
img = img.transpose((2, 0, 1))[::-1]
img = np.ascontiguousarray(img)

# テンソル化 -> 正規化
img = torch.from_numpy(img)
img = img.half() if fp16 else img.float()
img /= 255
img = img.unsqueeze(0)



#---- 推論 -----
img = img.cpu().numpy()
output_name = session.get_outputs()[0].name
input_name = session.get_inputs()[0].name

# yのshape:[データ数、bbox数, メタ情報]
# メタ情報:[x中心, y中心, bboxの幅, bboxの高さ, 信頼スコア、 押印の確率, 未押印の確率]
y = session.run([output_name], {input_name: img})[0]
pred = torch.from_numpy(y) # テンソル化

# NMS処理
pred = non_max_suppression(pred, conf_thres, iou_thres, agnostic_nms, max_det=max_det)

In [211]:
pred

[tensor([[483.87460, 543.45117, 529.17871, 598.46057,   0.95847,   0.00000],
         [389.51276,  70.15652, 432.96857, 127.43932,   0.94714,   0.00000],
         [431.27887,  69.81499, 474.19971, 127.03190,   0.94670,   1.00000],
         [473.34045,  67.55436, 515.70135, 125.63311,   0.94446,   1.00000]])]

## non_max_suppression関数の実装

In [207]:
def non_max_suppression(
    prediction, 
    conf_thres=0.25,
    iou_thres=0.45,
    agnostic=False,
    labels=(),
    max_det=300,
    nm=0,
):
    """
    
    YOLOの予測結果をNMS処理する。

    Parameters
    --------------
    prediction : (tensor) テンソル化したyoloが予測した結果
    conf_thres : (float) バウンディングボックス(bbox)の信頼度スコアの閾値
    iou_thres : (float) IOUの閾値
    agnostic : (Bool) 重なった異なるクラスのbboxを同一のbboxにするか
    labels : (tuple or list-like) 画像内のラベルの情報
    max_det : (int) 最大のbbox数
    nm : (int) マスク数
    
    Return
    -----------
    output : (tensor) NMS処理後のbbox情報を含むテンソル、[x1, y1, x2, y2, クラス確率, cls_id]のカラムに変換されている
                      x1,y1はbboxの左上のx,y座標、x2, y2は右下のx,y座標を表す
                      
    """
    batch_size = prediction.shape[0] # バッチサイズ
    num_class = prediction.shape[2] - nm - 5 # クラス数
    bool_cnf = prediction[..., 4] > conf_thres # bbox毎に信頼度スコアがconf_thresより大きいかのbool
    
    max_wh = 7680  # 最大のbboxの幅、高さ
    max_nms = 30000  # torchvision.ops.nms()のための最大のbbox数
    time_limit = 0.5 + 0.05 * batch_size  # タイムリミット(s)
    
    start = time.time()
    mi = 5 + num_class
    output = [torch.zeros((0, 6 + nm), device=prediction.device)] * batch_size
    
    
    for idx, x in enumerate(prediction):
    
        # conf_thresより大きい信頼度スコアを持つbboxsを抽出
        x = x[bool_cnf[idx]]
    
        # labelsを持っていた時の処理
        if labels and len(labels[idx]):
                lb = labels[idx]
                v = torch.zeros((len(lb), num_class + nm + 5), device=x.device)
                v[:, :4] = lb[:, 1:5]  # box
                v[:, 4] = 1.0  # conf
                v[range(len(lb)), lb[:, 0].long() + 5] = 1.0  # cls
                x = torch.cat((x, v), 0)
            
    
        # フィルター後のbboxがない場合
        if not x.shape[0]:
            continue
            
        x[:, 5:] *= x[:, 4:5] # 信頼度スコア*各cls確率
        box = xywh2xyxy(x[:, :4]) # [x1, y1, x2, y2]:bboxの左上と右下の座標に変換
        mask = x[:, mi:] # maskがなければ[]
    
        cls_prob, cls_id = x[:, 5:mi].max(1, keepdim=True) # 各bboxで確率の高いclsの値とcls_idを返す
    
        # [x1, y1, x2, y2, clsの確率, cls_id]のカラムの順に結合
        # clsの確率(本来のprob*信頼度スコア)がconf_thresより高いもののみ抽出
        x = torch.cat((box, cls_prob, cls_id.float(), mask), axis=1)[cls_prob.view(-1) > conf_thres]
    
        
        n = x.shape[0]
        if not n:
            continue
    
        # cls確率をキーにして降順に並び替え、且つ max_nmsを超えないようにする
        x = x[x[:, 4].argsort(descending=True)[:max_nms]] 
    
        # 異なるクラスのbboxを区別して評価するため、agnosticでクラス固有のbboxを作成
        c = x[:, 5:6] * (0 if agnostic else max_wh)
        boxes, scores = x[:, :4] + c, x[:, 4]
    
        # nms
        selected_idx = torchvision.ops.nms(boxes, scores, iou_thres)
        selected_idx = selected_idx[:max_det]

        output[idx] = x[selected_idx]
        finish = time.time()
        if (finish - start) > time_limit:
            break
            
    return output


## 'xywh2xyxy

In [208]:
def xywh2xyxy(x):
    """
    [x中心、y中心、bboxの幅, bboxの高さ] -> [bboxの左上のx、bboxの左上のy, bboxの右下のx, bboxの右下のy]
    
    """
    y = x.clone()
    y[..., 0] = x[..., 0] - x[..., 2] / 2  # bboxの左上のx
    y[..., 1] = x[..., 1] - x[..., 3] / 2  # bboxの左上のy
    y[..., 2] = x[..., 0] + x[..., 2] / 2  # bboxの右下のx
    y[..., 3] = x[..., 1] + x[..., 3] / 2  # bboxの右下のy
    return y

# letterbox関数の実装

In [68]:
def letterbox(
    img,
    new_shape=(640, 640),
    color=(0, 0, 0),
    scaleup=True,
):
    """
    指定されたサイズに画像をリサイズを行う。

    Parameters
    --------------
    img : (ndarray) 画像のndarray配列
    new_shape : (tuple) リサイズしたい画像サイズ
    color : (tuple:(B, G, R)) パディングの色
    scaleup : (Bool) スケールアップを行う場合はTrue

    Return
    -----------
    padded_img : (ndarray) リサイズされた画像のndarray配列
    ratio : (tuple リサイズした比率
    (dw, dh) : (int:(横、高さ)) パディングした数値(横、高さ)
    """
    
    # アスペクト比を保った画像のリサイズ
    ori_shape = img.shape[:2] # [H, W, C] -> [H, W]
    r = min(new_shape[0] / ori_shape[0], new_shape[1] / ori_shape[1])
    
    # スケールアップなしの場合
    if not scaleup:
        r = min(r, 1.0)
    
    # アスペクト比を保ったまま、リスケール
    ratio = (r, r)
    new_unpad = (round(ori_shape[1] * r), round(ori_shape[0] * r)) # cv.resizeのため[W, H]にする
    
    # パディングする領域を算出
    dw, dh = new_shape[1] - new_unpad[0], new_shape[0] - new_unpad[1]
    dw, dh = dw / 2, dh / 2

    # dw, dfが.5の時の対応 -> この微調整をしないとsession.run()時のinputのshapeが合わなくなる
    top, bottom = round(dh - 0.1), round(dh + 0.1)
    left, right = round(dw - 0.1), round(dw + 0.1)
    
    # 画像のリサイズ
    if new_shape[::-1] != new_unpad:
        img = cv2.resize(img, new_unpad, interpolation=cv2.INTER_LINEAR)
        
    padded_img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)

    return padded_img, ratio, (dw, dh)